In [1]:
import argparse
import logging
import pandas as pd
import random
import os
import numpy as np
from process import input_data, modeling, preprocess
from sklearn.model_selection import train_test_split

import torch
import torch.nn as nn
from pytorch_tabnet.tab_model import TabNetClassifier

In [2]:
log_name='1'
data, var_list, num_var, obj_var = input_data.Data_load('storage/data/churn.csv', log_name).read_data()
df = preprocess.Preprocessing(log_name, data, var_list, num_var, obj_var, target='churn', unique_id = 'state' ).get_df()
#mm = modeling.Modeling(log_name, df, obj_var = obj_var, target='churn', unique_id = 'state', model_type='auto', OVER_SAMPLING=False, HPO=False)
    

결측치 처리에 문제가 발생하였습니다
Traceback (most recent call last):
  File "C:\Users\LDCC\Desktop\eco_code\automl\process\preprocess.py", line 47, in na_preprocess
    obj_data.fillna("NaN", inplace=True)
  File "C:\Users\LDCC\anaconda3\envs\practice\lib\site-packages\pandas\util\_decorators.py", line 311, in wrapper
    return func(*args, **kwargs)
  File "C:\Users\LDCC\anaconda3\envs\practice\lib\site-packages\pandas\core\frame.py", line 5182, in fillna
    downcast=downcast,
  File "C:\Users\LDCC\anaconda3\envs\practice\lib\site-packages\pandas\core\generic.py", line 6383, in fillna
    value=value, limit=limit, inplace=inplace, downcast=downcast
  File "C:\Users\LDCC\anaconda3\envs\practice\lib\site-packages\pandas\core\internals\managers.py", line 411, in fillna
    "fillna", value=value, limit=limit, inplace=inplace, downcast=downcast
  File "C:\Users\LDCC\anaconda3\envs\practice\lib\site-packages\pandas\core\internals\managers.py", line 327, in apply
    applied = getattr(b, f)(**kwargs)
  F

In [3]:
data  =pd.read_csv('storage/data/churn2.csv')
data.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [13]:
np.sum(data.isnull().sum())

0

In [10]:
data.fillna('NaN', inplace=True)

In [11]:
mm.best_model.predict_proba(mm.X_train.values)

array([[0.1947415 , 0.8052585 ],
       [0.97269611, 0.02730389],
       [0.9906444 , 0.0093556 ],
       ...,
       [0.98844503, 0.01155497],
       [0.9813635 , 0.0186365 ],
       [0.97941427, 0.02058573]])

In [12]:
df = df.drop(['state'], axis=1)
target = 'churn'
unique_id = 'state'

In [13]:
df_y = pd.DataFrame(df.loc[:,target])
df_x = df.drop(target, axis=1)
X_train, X_test, y_train, y_test = train_test_split(df_x, df_y, test_size=0.2, random_state=20)            

In [14]:
nuinque = df.nunique()
types = df.dtypes

categorical_columns = []
categorical_dims = {}
target = 'churn'

for col in obj_var:
    categorical_columns.append(col)
    categorical_dims[col] = df[col].nunique()

features = [col for col in df.columns if col not in [target]]
cat_idxs = [i for i, f in enumerate(features) if f in obj_var]
cat_dims = [categorical_dims[f] for i, f in enumerate(features) if f in categorical_columns]

tabnet_param = {'cat_idxs' : cat_idxs, 'cat_dims':cat_dims}

In [15]:
len(features)

19

In [16]:
X_train_values = X_train.values
y_train_values = y_train.values.reshape(-1,)
X_test_values = X_test.values
y_test_values = y_test.values.reshape(-1,)


parameters = {'mask_type': 'entmax', 'n_da': 8, 'n_steps': 3, 'gamma': 1.3, 'n_shared': 2,'lambda_sparse': 1e-3, 'patience': 10, 'epochs': 2}


tab=TabNetClassifier(cat_idxs=cat_idxs, cat_dims=cat_dims, cat_emb_dim=10, 
         optimizer_fn = torch.optim.Adam, optimizer_params=dict(lr=1e-2),
         n_d = parameters['n_da'], n_a = parameters['n_da'], n_steps = parameters['n_steps'], n_shared = parameters['n_shared'], 
         lambda_sparse = parameters['lambda_sparse'], gamma = parameters['gamma'],
         scheduler_params={'step_size':50},
         scheduler_fn = torch.optim.lr_scheduler.StepLR, mask_type = parameters['mask_type']) 

tab.fit(X_train=X_train_values, y_train=y_train_values,
       eval_set=[(X_train_values, y_train_values), (X_test_values, y_test_values)], eval_name=['train', 'valid'], eval_metric=['auc'],
        max_epochs=parameters['epochs'], patience=parameters['patience'], 
        batch_size=1024, virtual_batch_size =128, num_workers=1, weights=1, drop_last=False,)# loss_fn=focal_loss)

y_pred_proba = tab.predict_proba(X_test_values)[:,1]
y_pred = tab.predict(X_test_values)

Device used : cpu
epoch 0  | loss: 0.75379 | train_auc: 0.57649 | valid_auc: 0.59507 |  0:00:04s
epoch 1  | loss: 0.67075 | train_auc: 0.61691 | valid_auc: 0.63021 |  0:00:10s
Stop training because you reached max_epochs = 2 with best_epoch = 1 and best_valid_auc = 0.63021
Best weights from best epoch are automatically used!


In [42]:
fi = tab.feature_importances_
#fi_df = pd.DataFrame(fi.reshape(1,-1), columns = X_test.columns)
fi_df = pd.DataFrame(fi, index=X_test.columns).sort_values(0, ascending=False)

In [43]:
fi_df

,0
international_plan,0.347357
phone_number,0.198768
voice_mail_plan,0.127304
total_eve_charge,0.034888
total_day_charge,0.032869
total_eve_minutes,0.031319
total_eve_calls,0.029358
area_code,0.027111
total_intl_charge,0.023793
total_day_calls,0.022806


In [17]:
explain, _ = tab.explain(X_test_values)

In [18]:
instance_ = pd.DataFrame(explain, columns = X_test.columns)
instance_importance = instance_.div(instance_.sum(axis=1), axis=0)

In [19]:
instance_importance

,phone_number,international_plan,voice_mail_plan,total_day_minutes,total_day_charge,total_eve_minutes,total_eve_charge,total_night_minutes,total_night_charge,total_intl_minutes,total_intl_charge,account_length,area_code,number_vmail_messages,total_day_calls,total_eve_calls,total_night_calls,total_intl_calls,customer_service_calls
0,0.181052,0.241551,0.159949,0.007675,0.019296,0.019812,0.042632,0.003245,0.029736,0.002126,0.071907,0.048460,0.034083,0.008478,0.011169,0.071565,0.019078,0.028184,0.000000e+00
1,0.155975,0.263664,0.198614,0.001899,0.019960,0.045698,0.058462,0.001042,0.055102,0.000000,0.038040,0.000000,0.049193,0.022941,0.045538,0.011743,0.000454,0.031676,0.000000e+00
2,0.162120,0.293173,0.128430,0.006088,0.041130,0.010863,0.027474,0.005896,0.030546,0.003374,0.056768,0.054988,0.035247,0.008286,0.027255,0.064659,0.038402,0.004579,7.215821e-04
3,0.151877,0.115682,0.149570,0.000718,0.003120,0.466021,0.019289,0.000495,0.005386,0.003338,0.015313,0.006471,0.005985,0.010162,0.007325,0.009790,0.005904,0.023554,5.845997e-07
4,0.148265,0.793818,0.008396,0.000945,0.000625,0.000004,0.000329,0.000000,0.001247,0.024041,0.000150,0.017423,0.000000,0.001715,0.000434,0.000905,0.000005,0.000964,7.336420e-04
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
662,0.162562,0.598136,0.141638,0.001318,0.002915,0.000000,0.000006,0.000017,0.000182,0.000000,0.026778,0.000000,0.000752,0.000344,0.029524,0.012747,0.000280,0.022799,0.000000e+00
663,0.197966,0.400015,0.079315,0.000000,0.053149,0.013698,0.003634,0.000060,0.015197,0.000000,0.025138,0.087277,0.002997,0.000000,0.028215,0.056522,0.031503,0.005313,0.000000e+00
664,0.184319,0.318721,0.186311,0.009818,0.027972,0.002192,0.078144,0.016496,0.025057,0.009399,0.031052,0.002342,0.045153,0.002981,0.024429,0.012077,0.021662,0.001875,0.000000e+00
665,0.338250,0.059246,0.126666,0.006915,0.000000,0.082124,0.070730,0.042893,0.002332,0.224077,0.002280,0.029559,0.002136,0.000000,0.000134,0.000000,0.000000,0.012291,3.670348e-04
